# Testing monetization workflow with `.mif` and mapping files

In [1]:
from premise.data_collection import IAMDataCollection
from premise.activity_maps import InventorySet
import brightway2 as bw
import pandas as pd
import xarray as xr

## 1. Load REMIND reporting file

As an example, we'll work with production volumes.

In [3]:
iam_data = IAMDataCollection("remind", "default", 2030, "../remind_data/", None)
iam_data.production_volumes

<xarray.DataArray 'value' (region: 13, variables: 48, year: 19)>
array([[[5.96300000e-04, 7.60100000e-04, 7.53900000e-04, ...,
         7.00000000e-07, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.68424000e-02, 5.46379000e-02, 5.40414000e-02, ...,
         8.00000000e-07, 0.00000000e+00, 0.00000000e+00],
        ...,
        [7.14992000e+00, 5.54636670e+00, 4.93317990e+00, ...,
         1.46478307e+01, 1.60704899e+01, 1.53692196e+01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [4.74387000e-02, 5.53980000e-02, 5.47953000e-02, ...,
         1.28798100e-01, 1.28802100e-01, 1.28802100e-01]],

       [[0.00000000e+00, 1.77089000e-02, 9.65938000e-02, ...,
         2.00000000e-07, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [2.95090000e-02, 3.96727000e-02, 9.10959000e-02, ...,
         3.00000000e-07, 0.00000000e+00, 0.00000000e+00],
...
        [5.19730000e+01, 4.98313800e+01, 4.32051948e+01, ...,
         8.80261997e+01, 9.76852922e+01, 9.43328453e+01],
        [0.00000000e+00, 1.43913000e-02, 0.00000000e+00, ...,
         8.31141900e-01, 8.95337900e-01, 8.95337900e-01],
        [2.68090600e-01, 3.03268200e-01, 3.25662100e-01, ...,
         3.14828600e-01, 3.14828800e-01, 3.14828800e-01]],

       [[1.82647900e-01, 3.12584900e-01, 4.81587600e-01, ...,
         7.30000000e-06, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [6.63479500e-01, 8.81074600e-01, 1.14710380e+00, ...,
         7.40000000e-06, 0.00000000e+00, 0.00000000e+00],
        ...,
        [3.13925790e+02, 3.47858923e+02, 2.65295198e+02, ...,
         9.63537328e+02, 9.63594052e+02, 8.09446947e+02],
        [8.00000000e-07, 1.01057500e-01, 4.41331000e-02, ...,
         4.34117990e+00, 6.29010340e+00, 6.59171230e+00],
        [8.46126600e-01, 1.09260190e+00, 1.58455830e+00, ...,
         3.20421040e+00, 3.31786000e+00, 3.31786000e+00]]])
Coordinates:
  * region     (region) object 'CAZ' 'CHA' 'EUR' 'IND' ... 'SSA' 'USA' 'World'
  * variables  (variables) <U38 'Biomass CHP' ... 'biomass - residual'
  * year       (year) int64 2005 2010 2015 2020 2025 ... 2100 2110 2130 2150

## 2. Load database

In [5]:
from premise import NewDatabase
from premise.activity_maps import get_mapping
bw.projects.set_current("test")

In [6]:
ndb = NewDatabase(
    scenarios = [
        {"model":"remind", "pathway":"SSP2-Base", "year":2035},
    ],
    source_db="ecoinvent 3.7.1 cutoff",
    source_version="3.7.1",
    key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
)

premise v.(1, 3, 2)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

## 3. Get a mapping file

In [16]:
from premise.data_collection import IAM_ELEC_VARS as fp
mapping = get_mapping(fp, "iam_aliases")
premise2rmnd = {}
for k, v in mapping.items():
    if "remind" in v.keys():
        premise2rmnd[k] = v["remind"]
premise2rmnd

{'Biomass CHP': 'SE|Electricity|Biomass|++|Combined Heat and Power w/o CC',
 'Biomass IGCC CCS': 'SE|Electricity|Biomass|++|Gasification Combined Cycle w/ CC',
 'Biomass IGCC': 'SE|Electricity|Biomass|++|Gasification Combined Cycle w/o CC',
 'Coal PC': 'SE|Electricity|Coal|++|Pulverised Coal w/o CC',
 'Coal IGCC': 'SE|Electricity|Coal|++|Gasification Combined Cycle w/o CC',
 'Coal PC CCS': 'SE|Electricity|Coal|++|Pulverised Coal w/ CC',
 'Coal IGCC CCS': 'SE|Electricity|Coal|++|Gasification Combined Cycle w/ CC',
 'Coal CHP': 'SE|Electricity|Coal|++|Combined Heat and Power w/o CC',
 'Gas OC': 'SE|Electricity|Gas|++|Gas Turbine',
 'Gas CC': 'SE|Electricity|Gas|++|Combined Cycle w/o CC',
 'Gas CHP': 'SE|Electricity|Gas|++|Combined Heat and Power w/o CC',
 'Gas CC CCS': 'SE|Electricity|Gas|++|Combined Cycle w/ CC',
 'Geothermal': 'SE|Electricity|+|Geothermal',
 'Hydro': 'SE|Electricity|+|Hydro',
 'Nuclear': 'SE|Electricity|+|Nuclear',
 'Oil ST': 'SE|Electricity|Oil|w/o CC',
 'Solar CSP': 

In [18]:
InvSet = InventorySet(ndb.database)

In [22]:
mapping = InvSet.generate_powerplant_map()
mapping

{'Biomass CHP': {'heat and power co-generation, wood chips, 2000 kW',
  'heat and power co-generation, wood chips, 2000 kW, state-of-the-art 2014',
  'heat and power co-generation, wood chips, 6667 kW',
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014',
  'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014, label-certified'},
 'Biomass CHP CCS': {'electricity production, at co-generation power plant/wood, post, pipeline 200km, storage 1000m',
  'electricity production, at co-generation power plant/wood, post, pipeline 400km, storage 3000m'},
 'Biomass ST': {'electricity production, at wood burning power plant 20 MW, truck 25km, no CCS'},
 'Biomass IGCC CCS': {'electricity production, at BIGCC power plant, pre, pipeline 200km, storage 1000m',
  'electricity production, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m'},
 'Biomass IGCC': {'electricity production, at BIGCC power plant, no CCS'},
 'Coal PC':

## 4. Do LCIA

As a simplification, if there are several ecoinvent activities per REMIND one, we use an averaged demand in the LCA.

In [20]:
methods =  [
    ('ReCiPe Midpoint (H) V1.13', 'agricultural land occupation', 'ALOP'),
    ('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100'),
    ('ReCiPe Midpoint (H) V1.13', 'fossil depletion', 'FDP'),
    ('ReCiPe Midpoint (H) V1.13', 'freshwater ecotoxicity', 'FETPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'freshwater eutrophication', 'FEP'),
    ('ReCiPe Midpoint (H) V1.13', 'human toxicity', 'HTPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'ionising radiation', 'IRP_HE'),
    ('ReCiPe Midpoint (H) V1.13', 'marine ecotoxicity', 'METPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'marine eutrophication', 'MEP'),
    ('ReCiPe Midpoint (H) V1.13', 'metal depletion', 'MDP'),
    ('ReCiPe Midpoint (H) V1.13', 'natural land transformation', 'NLTP'),
    ('ReCiPe Midpoint (H) V1.13', 'ozone depletion', 'ODPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'particulate matter formation', 'PMFP'),
    ('ReCiPe Midpoint (H) V1.13', 'photochemical oxidant formation', 'POFP'),
    ('ReCiPe Midpoint (H) V1.13', 'terrestrial acidification', 'TAP100'),
    ('ReCiPe Midpoint (H) V1.13', 'terrestrial ecotoxicity', 'TETPinf'),
    ('ReCiPe Midpoint (H) V1.13', 'urban land occupation', 'ULOP'),
    ('ReCiPe Midpoint (H) V1.13', 'water depletion', 'WDP')
]

In [42]:
eidb = bw.Database("ecoinvent_remind_default_2030")

locations = {}
for k, v in mapping.items():
    locs = []
    for name in v:
        for act in eidb:
            if name in act["name"]:
                locs.append(act["location"])
    locations[k] = locs
locations

{'Biomass CHP': ['CH',
  'CH',
  'CH',
  'CH',
  'KR',
  'AU',
  'AU',
  'DE',
  'CA-AB',
  'US-HICC',
  'SE',
  'US-FRCC',
  'DK',
  'BG',
  'CA-NS',
  'CH',
  'DK',
  'CA-ON',
  'GR',
  'US-WECC',
  'US-HICC',
  'US-NPCC',
  'MT',
  'JP',
  'PL',
  'RO',
  'CA-NS',
  'HR',
  'PT',
  'CL',
  'CA-AB',
  'LV',
  'RoW',
  'EE',
  'CH',
  'MT',
  'IT',
  'US-MRO',
  'SE',
  'AT',
  'LU',
  'ZA',
  'ES',
  'US-RFC',
  'CL',
  'FR',
  'GR',
  'CN-GD',
  'CA-QC',
  'CH',
  'HU',
  'IE',
  'CZ',
  'US-WECC',
  'IE',
  'BG',
  'SK',
  'FR',
  'PL',
  'LV',
  'CY',
  'US-SERC',
  'US-TRE',
  'RoW',
  'RU',
  'CA-BC',
  'RO',
  'DE',
  'IT',
  'LU',
  'JP',
  'US-NPCC',
  'PT',
  'US-RFC',
  'CA-PE',
  'AT',
  'US-TRE',
  'NO',
  'US-SERC',
  'RU',
  'CA-QC',
  'BE',
  'NL',
  'GB',
  'CZ',
  'NL',
  'US-MRO',
  'CH',
  'HU',
  'HR',
  'IN-TN',
  'US-ASCC',
  'FI',
  'SK',
  'GB',
  'IN-TN',
  'ES',
  'FI',
  'CA-ON',
  'SI',
  'CN-GD',
  'US-ASCC',
  'CY',
  'US-FRCC',
  'CA-PE',
  'CN-SH',
  '

In [23]:
lcia_scores = {}


for k, v in mapping.items():
    N = len(v)
    demand = {}
    for act in v:
        demand[act] = 1/N
    lca = bw.LCA(demand)
    lca.lci()

    scores = {}
    for m in methods:
        lca.switch_method(m)
        lca.lcia()
        scores[m[1]] = lca.score

    lcia_scores[k] = scores

KeyError: 'h'